In [1]:
import regex as re
import pandas as pd
from src.utils.preprocessing import *

%load_ext autoreload
%autoreload 2

In [2]:
def find_difference(string1, string2):
    lines1 = [line.strip().lower() for line in string1.split('\n')]
    lines2 = [line.strip().lower() for line in string2.split('\n')]
    
    toappend = []
    toremove = []
    pattern = r'review of systems: (.*?)$'

    for line in lines1:
        if re.findall(pattern, line, re.MULTILINE | re.IGNORECASE):
            toappend.append("review of systems:")
            toappend.append(line.split("review of systems: ")[-1])
            toremove.append(line)

    for line in toremove:
        lines1.remove(line)

    lines1.extend(toappend)

    diff_lines = [line for line in lines1 if line not in lines2]
    diff_string = '\n'.join(diff_lines)
    
    return diff_string

In [66]:
discharge_df = pd.read_csv('data/version1.4_ours/valid/discharge.csv.gz', compression='gzip')
target_df = pd.read_csv('data/version1.4_ours/valid/discharge_target.csv.gz', compression='gzip')

In [67]:
combined_df = build_combined_discharge(discharge_df, target_df)
combined_df['clean_discharge'] = combined_df[['text','brief_hospital_course', 'discharge_instructions']] \
    .progress_apply(axis = 1,func = lambda x: remove_bhc_di(x['text'], x['brief_hospital_course'], x['discharge_instructions'], 'bhc'))

100%|██████████| 14719/14719 [00:02<00:00, 7178.60it/s]


In [68]:
originals = combined_df['clean_discharge']

In [70]:
valid_data_reconstructed = []

for i in tqdm(range(len(combined_df))):
    if i > 12800:
        first_valid = originals[i]
        start_idx = 0
        reconstructed = ''
        for section in section_to_next_section:
            exctaction, start_idx = extract_section(first_valid, section, start_idx)
            reconstructed += exctaction
        valid_data_reconstructed.append(reconstructed)

combined_df['reconstructed'] = valid_data_reconstructed
reconstructed = valid_data_reconstructed
#print(valid_data['reconstructed'][0])

  0%|          | 0/14719 [00:00<?, ?it/s]

100%|██████████| 14719/14719 [00:03<00:00, 3919.97it/s]  


ValueError: Length of values (1918) does not match length of index (14719)

In [15]:
for i in range(100,200):#len(reconstructed)):
    print('----------------------------------------------------------')
    print('index:', i)
    difference = find_difference(first_valid_gold[i], reconstructed[i])
    print(difference)

----------------------------------------------------------
index: 100
name:  ___                 unit no:   ___
admission date:  ___              discharge date:   ___
date of birth:  ___             sex:   f
attending: ___.
followup instructions:
___
----------------------------------------------------------
index: 101
name:  ___                 unit no:   ___
admission date:  ___              discharge date:   ___
date of birth:  ___             sex:   f
attending: ___.
followup instructions:
----------------------------------------------------------
index: 102
name:  ___                 unit no:   ___
admission date:  ___              discharge date:   ___
date of birth:  ___             sex:   m
attending: ___
___ complaint:
facility:
followup instructions:
----------------------------------------------------------
index: 103
name:  ___                   unit no:   ___
admission date:  ___              discharge date:   ___
date of birth:  ___             sex:   m
attending: ___.
f

In [38]:
text = """Name:  ___                      Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.
 
Major Surgical or Invasive Procedure:
none

attach
 
Pertinent Results:
LABS
==================

___ 05:05AM BLOOD WBC-3.8* RBC-2.41* Hgb-8.8* Hct-27.6* 
MCV-115* MCH-36.5* MCHC-31.9* RDW-14.9 RDWSD-63.5* Plt ___
___ 05:20AM BLOOD WBC-3.9* RBC-2.44* Hgb-8.8* Hct-27.9* 
MCV-114* MCH-36.1* MCHC-31.5* RDW-15.1 RDWSD-63.7* Plt ___
___ 03:20PM BLOOD WBC-4.6 RBC-2.42* Hgb-8.8* Hct-27.6* 
MCV-114* MCH-36.4* MCHC-31.9* RDW-15.0 RDWSD-63.7* Plt ___
___ 06:00AM BLOOD WBC-6.2 RBC-2.52* Hgb-9.1* Hct-28.6* 
MCV-114* MCH-36.1* MCHC-31.8* RDW-15.1 RDWSD-63.5* Plt ___
___ 05:42PM BLOOD WBC-8.3 RBC-2.73* Hgb-10.1* Hct-29.9* 
MCV-110* MCH-37.0* MCHC-33.8 RDW-15.3 RDWSD-61.2* Plt Ct-81*
___ 05:42PM BLOOD Neuts-84.2* Lymphs-7.6* Monos-7.8 
Eos-0.0* Baso-0.2 Im ___ AbsNeut-6.94* AbsLymp-0.63* 
AbsMono-0.64 AbsEos-0.00* AbsBaso-0.02
___ 05:42PM BLOOD ___ PTT-32.8 ___
___ 05:05AM BLOOD Glucose-96 UreaN-5* Creat-0.6 Na-143 
K-3.7 Cl-104 HCO3-28 AnGap-11
___ 05:20AM BLOOD Glucose-97 UreaN-5* Creat-0.6 Na-142 
K-3.6 Cl-102 HCO3-26 AnGap-14
___ 06:00AM BLOOD Glucose-92 UreaN-6 Creat-0.6 Na-141 K-3.7 
Cl-100 HCO3-27 AnGap-14
___ 06:18AM BLOOD Glucose-92 UreaN-7 Creat-0.6 Na-138 
K-3.1* Cl-98 HCO3-27 AnGap-13
___ 05:42PM BLOOD Glucose-117* UreaN-9 Creat-0.6 Na-137 
K-3.6 Cl-96 HCO3-26 AnGap-15
___ 05:20AM BLOOD ALT-208* AST-199* AlkPhos-138* 
TotBili-0.3
___ 06:00AM BLOOD ALT-298* AST-383* AlkPhos-158* 
TotBili-0.3
___ 06:18AM BLOOD ALT-320* AST-492* AlkPhos-153* 
TotBili-0.6
___ 05:42PM BLOOD ALT-349* AST-651* AlkPhos-193* 
TotBili-0.7
___ 06:18AM BLOOD Lipase-122*
___ 05:42PM BLOOD Lipase-160*
___ 05:42PM BLOOD cTropnT-<0.01
___ 05:42PM BLOOD proBNP-246*
___ 05:05AM BLOOD Calcium-9.0 Phos-2.6* Mg-1.6
___ 05:20AM BLOOD Calcium-9.2 Phos-2.6* Mg-1.3*
___ 06:00AM BLOOD Calcium-9.2 Phos-3.0 Mg-1.6 Iron-79
___ 06:18AM BLOOD Albumin-3.3* Calcium-8.8 Phos-2.5* Mg-2.1
___ 05:42PM BLOOD Albumin-4.0 Calcium-9.3 Phos-1.8* Mg-1.2*
___ 06:00AM BLOOD calTIBC-160* Ferritn-4183* TRF-123*
___ 06:00AM BLOOD AMA-NEGATIVE Smooth-NEGATIVE
___ 06:00AM BLOOD ___ Titer-1:40*
___ 06:00AM BLOOD IgG-556* IgA-138 IgM-93
___ 08:43AM BLOOD ASA-NEG Ethanol-NEG Acetmnp-NEG 
Tricycl-NEG
___ 05:56PM BLOOD Lactate-1.0 Creat-0.6
___ 04:49PM URINE Color-DkAmb* Appear-Hazy* Sp ___
___ 04:49PM URINE Blood-NEG Nitrite-NEG Protein-100* 
Glucose-NEG Ketone-10* Bilirub-SM* Urobiln-4* pH-6.5 Leuks-LG*
___ 04:49PM URINE RBC-3* WBC-44* Bacteri-FEW* Yeast-NONE 
Epi-12 TransE-<1
___ 04:49PM URINE CastHy-10*
___ 04:49PM URINE AmorphX-RARE*
___ 04:49PM URINE Mucous-MANY*
___ 11:49PM STOOL CDIFPCR-POS* CDIFTOX-NEG

MICRO 
============
__________________________________________________________
___ 3:40 pm BLOOD CULTURE

   Blood Culture, Routine (Pending):    No growth to date. 
__________________________________________________________
___ 3:20 pm BLOOD CULTURE      Source: Line-port. 

   Blood Culture, Routine (Pending):    No growth to date. 
__________________________________________________________
___ 11:49 pm STOOL     CONSISTENCY: NOT APPLICABLE
      Source: Stool. 

                            **FINAL REPORT ___

   FECAL CULTURE (Final ___:    NO SALMONELLA OR SHIGELLA 
FOUND. 

   CAMPYLOBACTER CULTURE (Final ___:    NO CAMPYLOBACTER 
FOUND. 
__________________________________________________________
___ 4:49 pm URINE

                            **FINAL REPORT ___

   URINE CULTURE (Final ___: 
      MIXED BACTERIAL FLORA ( >= 3 COLONY TYPES), CONSISTENT 
WITH SKIN
      AND/OR GENITAL CONTAMINATION. 

IMAGING
=================
CXR (___):
No evidence of acute cardiopulmonary disease or injury.

CT Abdomen/Pelvis w/ Contrast (___):
Findings consistent with active colitis from the mid transverse 
colon through sigmoid.  Fatty infiltration of the liver.  No 
evidence of acute injury. 

DISCHARGE PHYSICAL EXAM
=========================
PHYSICAL EXAM:  
___ 1328 Temp: 99.0 PO BP: 134/71 HR: 97 RR: 18 O2 sat: 
100% O2 delivery: RA 

GENERAL: NAD  
HEENT: AT/NC, Sclerae anicteric, MMM  
NECK: Supple, no LAD  
CV: NR, RR. Normal S1/S2, no murmurs, gallops, or rubs  
PULM: CTAB, no wheezes, rales, rhonchi, breathing comfortably
without use of accessory muscles  
CHEST: Tender to palpation over left lower ribs
ABD: abdomen soft, nondistended, nontender in all quadrants, no
rebound/guarding, no hepatosplenomegaly  
EXT: No ___ edema  
SKIN: Warm and well perfused, no excoriations or lesions, no
rashes  
NEURO: Alert, moving all 4 extremities with purpose, face
symmetric  
ACCESS: Port
 


 
Medications on Admission:
The Preadmission Medication list is accurate and complete.
1. Acetaminophen 500 mg PO Q6H:PRN Pain - Mild/Fever 
2. Atorvastatin 20 mg PO QPM 
3. Multivitamins 1 TAB PO DAILY 
4. Omeprazole 20 mg PO DAILY 
5. Cyanocobalamin 1000 mcg PO DAILY 
6. Lidocaine-Prilocaine 1 Appl TP ASDIR for port access 
7. Ondansetron 8 mg PO Q8H:PRN Nausea/Vomiting - First Line 
8. Prochlorperazine 10 mg PO Q6H:PRN Nausea/Vomiting - Second 
Line 
9. Magnesium Oxide 400 mg PO DAILY 
10. Potassium Chloride (Powder) 20 mEq PO DAILY 
11. Gabapentin 100 mg PO TID 
12. Docusate Sodium 100 mg PO BID 

 
Discharge Medications:
1.  LOPERamide 2 mg PO QID:PRN severe diarrhea 
RX *loperamide 2 mg 2 mg by mouth three times a day Disp #*10 
Capsule Refills:*0 
2.  Acetaminophen 500 mg PO Q6H:PRN Pain - Mild/Fever  
3.  Atorvastatin 20 mg PO QPM  
4.  Cyanocobalamin 1000 mcg PO DAILY  
5.  Docusate Sodium 100 mg PO BID  
6.  Gabapentin 100 mg PO TID  
7.  Lidocaine-Prilocaine 1 Appl TP ASDIR for port access  
8.  Magnesium Oxide 400 mg PO DAILY  
9.  Multivitamins 1 TAB PO DAILY  
10.  Omeprazole 20 mg PO DAILY  
11.  Ondansetron 8 mg PO Q8H:PRN Nausea/Vomiting - First Line  
12.  Potassium Chloride (Powder) 20 mEq PO DAILY 
Hold for K >  
13.  Prochlorperazine 10 mg PO Q6H:PRN Nausea/Vomiting - Second 
Line  

 
Discharge Disposition:
Home With Service
 
Facility:
___
 
Discharge Diagnosis:
Ischemic colitis

 
Discharge Condition:
Mental Status: Clear and coherent.
Level of Consciousness: Alert and interactive.
Activity Status: Ambulatory - Independent.

 



 
Followup Instructions:
___"""
start_idx = 0
for section in section_to_next_section:
      print(f'======={section}=======')
      print(f"old start index: {start_idx}")
      extracted, idx = extract_section(text, section, start_idx)
      start_idx = idx
      print(extracted)
      print(f"new start index: {start_idx}")

=======sex=======
old start index: 0
Sex:
F


new start index: 149
=======service=======
old start index: 149
Service:
MEDICINE


new start index: 169
=======allergies=======
old start index: 169
Allergies:



new start index: 169
=======chief_complaint=======
old start index: 169
Chief Complaint:



new start index: 169
=======major_surgical_procedures=======
old start index: 169

none

attach
 
Pertinent Results:
LABS

___ 05:05AM BLOOD WBC-3.8* RBC-2.41* Hgb-8.8* Hct-27.6* 
MCV-115* MCH-36.5* MCHC-31.9* RDW-14.9 RDWSD-63.5* Plt ___
___ 05:20AM BLOOD WBC-3.9* RBC-2.44* Hgb-8.8* Hct-27.9* 
MCV-114* MCH-36.1* MCHC-31.5* RDW-15.1 RDWSD-63.7* Plt ___
___ 03:20PM BLOOD WBC-4.6 RBC-2.42* Hgb-8.8* Hct-27.6* 
MCV-114* MCH-36.4* MCHC-31.9* RDW-15.0 RDWSD-63.7* Plt ___
___ 06:00AM BLOOD WBC-6.2 RBC-2.52* Hgb-9.1* Hct-28.6* 
MCV-114* MCH-36.1* MCHC-31.8* RDW-15.1 RDWSD-63.5* Plt ___
___ 05:42PM BLOOD WBC-8.3 RBC-2.73* Hgb-10.1* Hct-29.9* 
MCV-110* MCH-37.0* MCHC-33.8 RDW-15.3 RDWSD-61.2* Plt Ct

ValueError: Alias Pertinent Results: found in section major_surgical_procedures.

In [21]:
s = 'Discharge Medications:'
e = 'Discharge Disposition:'

re.findall(rf'(?:\s{{2}}|\n+){s}(.*?)\n+{e}', text[start_idx:], re.DOTALL)

['\n ']